In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path ='/content/drive/MyDrive/Seventh Semester Project/Datasets/Train'
valid_path = '/content/drive/MyDrive/Seventh Semester Project/Datasets/Validate'
test_path = '/content/drive/MyDrive/Seventh Semester Project/Datasets/Test'
Img_size = [224, 224]
batch_size = 512

In [ ]:
## Data Augmentation

train_generator =ImageDataGenerator(rescale = 1/255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   rotation_range=30,
                                   fill_mode='nearest')
valid_generator = ImageDataGenerator(rescale= 1/255)
test_generator = ImageDataGenerator(rescale = 1/ 255)

In [ ]:
# Dataset 

train_datset = train_generator.flow_from_directory(directory=train_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

valid_datset = train_generator.flow_from_directory(directory=valid_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

test_datset = train_generator.flow_from_directory(directory=test_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

Found 10081 images belonging to 10 classes.
Found 2348 images belonging to 10 classes.
Found 2029 images belonging to 10 classes.


In [ ]:
resnet  = ResNet50(weights = 'imagenet', include_top=False, input_shape=Img_size +[3])
print('loaded model')

94781440/94765736 [==============================] - 0s 0us/step
loaded model


In [ ]:
for layer in resnet.layers:
  layer.trainable = False

In [ ]:
model=Sequential()
model.add(resnet)
#model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 225)               22579425  
                                                                 
Total params: 46,167,137
Trainable params: 22,579,425
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
opt=tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              optimizer=opt)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Here Early stopping callback is added for unnecessary running of epochs if model performance is poor 
filepath = '/content/drive/MyDrive/Seventh Semester Project/model/ResNet50_new1.h5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 10,
                              verbose = 1)

# for saving best model till execution
checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

callback_list = [earlystopping, checkpoint]

In [ ]:
result = model.fit(train_datset, 
                   validation_data=valid_datset, 
                   epochs = 100,
                   callbacks = callback_list)
                    #verbose= 1)

Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 8.4960 - accuracy: 0.1093  
Epoch 00001: val_accuracy improved from -inf to 0.12053, saving model to /content/drive/MyDrive/Seventh Semester Project/model/ResNet50_new1.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


20/20 [==============================] - 3067s 154s/step - loss: 8.4960 - accuracy: 0.1093 - val_loss: 7.2658 - val_accuracy: 0.1205
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 4.0844 - accuracy: 0.1357
Epoch 00002: val_accuracy did not improve from 0.12053
20/20 [==============================] - 239s 12s/step - loss: 4.0844 - accuracy: 0.1357 - val_loss: 3.3042 - val_accuracy: 0.0698
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 2.4176 - accuracy: 0.1902
Epoch 00003: val_accuracy did not improve from 0.12053
20/20 [==============================] - 235s 12s/step - loss: 2.4176 - accuracy: 0.1902 - val_loss: 2.3748 - val_accuracy: 0.1099
Epoch 4/100
20/20 [==============================] - ETA: 0s - loss: 2.0969 - accuracy: 0.2558
Epoch 00004: val_accuracy improved from 0.12053 to 0.17675, saving model to /content/drive/MyDrive/Seventh Semester Project/model/ResNet50_new1.h5
20/20 [==============================] - 238s 12s/step - loss: 